In [7]:
import pickle
import numpy as np
import heapq
# load the data
with open("pca_features.pkl", "rb") as file:
    track_ids = pickle.load(file)
    features_pca = pickle.load(file)

In [8]:
import rtree
print(rtree.__version__)

1.3.0


In [9]:
print(len(track_ids))
print((features_pca).shape)

11883
(237660, 154)


In [10]:
from rtree import index
from tqdm import tqdm

p = index.Property()
p.dimension = 154 
rtree_index = index.Index(properties=p)

total_vectors = len(track_ids) * 20  # Total de vectores para indexar
with tqdm(total=total_vectors, desc="Indexando vectores en R-Tree") as pbar:
    for song_idx, track_id in enumerate(track_ids):  # Iterando en las canciones
        for local_idx in range(20):  # Cada canción tiene 20 descriptores
            global_idx = song_idx * 20 + local_idx  # Índice global en features_pca
            vector = features_pca[global_idx]  # Obtener el vector local
            rtree_index.insert(global_idx, vector)  # Insertar en el R-Tree
            pbar.update(1)  # Incrementar la barra de progreso

Indexando vectores en R-Tree:   0%|          | 0/237660 [00:00<?, ?it/s]

Indexando vectores en R-Tree: 100%|██████████| 237660/237660 [12:12<00:00, 324.42it/s]


In [13]:
from collections import Counter
import numpy as np
def knn_top_R_tree(rtree_index, features_pca, track_ids, query_vectors, k=2):
    """
    Realiza una búsqueda KNN en un R-Tree para 20 vectores de una canción y retorna las `k` canciones más cercanas.

    Parámetros:
        rtree_index (rtree.index.Index): Índice R-Tree construido con los vectores PCA.
        features_pca (np.ndarray): Matriz de vectores PCA.
        track_ids (list): Lista de IDs de canciones, uno por canción.
        query_vectors (np.ndarray): 20 vectores de consulta en el espacio PCA (de una canción).
        k (int): Número de canciones más cercanas a retornar.

    Retorna:
        list: Lista de las `k` canciones más cercanas, en formato:
              [(track_id1, votos1), (track_id2, votos2), ...].
    """

    # Acumular resultados de los 20 vectores
    all_neighbors = []

    for query_vector in query_vectors:
        # Búsqueda KNN para cada vector de consulta
        nearest_neighbors = list(rtree_index.nearest(query_vector, num_results=20))  # Buscar 20 vecinos por vector ya que cada cancion tiene 20 vectores caracteristicos
        all_neighbors.extend(nearest_neighbors)

    # Asociar vecinos con sus canciones
    song_indices = [neighbor // 20 for neighbor in all_neighbors]  # Índices de canciones
    track_id_votes = [track_ids[song_idx] for song_idx in song_indices]  # Mapear a track_ids

    # Votación mayoritaria
    song_counts = Counter(track_id_votes)
    top_songs = song_counts.most_common(k)  # Seleccionar las `k` canciones más votadas

    return top_songs
def knn_top_lineal(cancionesc,track_ids,consultas, top_k):
    all_neighbors=[]
    
    for consulta in consultas:
        nearest_neighbors = [(-float('inf'), -1)] * 20 #20 caracteristicas
        for i, embedding in enumerate(cancionesc):
            dist = -np.linalg.norm(consulta - embedding)#consigo mi distancia euclidiana invertida para usar como max-heap
            if dist > nearest_neighbors[0][0]:
                heapq.heappop(nearest_neighbors)
                heapq.heappush(nearest_neighbors, (dist, i))
        nearest_neighbors = [i for dit, i in nearest_neighbors]
        all_neighbors.extend(nearest_neighbors)

    song_indices = [neighbor // 20 for neighbor in all_neighbors]
    track_id_votes = [track_ids[song_idx] for song_idx in song_indices]

    song_counts = Counter(track_id_votes)
    top_songs = song_counts.most_common(top_k)  

    return top_songs
def knn_busquedarango_lineal(cancionesc, track_ids, consultas,radio):
    all_neighbors = []
    distancias = [] 

    for consulta in consultas:
        nearest_neighbors = [(-float('inf'), -1)] * 20  # 20 características
        for i, embedding in enumerate(cancionesc):
            dist = -np.linalg.norm(consulta - embedding)  # Distancia euclidiana invertida para max-heap
            if dist > nearest_neighbors[0][0]:
                heapq.heappop(nearest_neighbors)
                heapq.heappush(nearest_neighbors, (dist, i))
        nearest_neighbors = [(dit, i) for dit, i in nearest_neighbors]
        all_neighbors.extend([i for dit, i in nearest_neighbors])
        distancias.extend([-dit for dit, i in nearest_neighbors])  

    song_indices = [neighbor // 20 for neighbor in all_neighbors]
    track_id_votes = [track_ids[song_idx] for song_idx in song_indices] 
    song_counts = Counter(track_id_votes)

    track_id_distancias = {}
    for idx, track_id in enumerate(track_id_votes):
        if track_id not in track_id_distancias:
            track_id_distancias[track_id] = []
        track_id_distancias[track_id].append(distancias[idx])
    distancias_canciones = [(song_id, votes, np.mean(track_id_distancias[song_id])) for song_id, votes in song_counts.most_common()]
    top_songs = []
    for i, (song_id, votes, avg_dist) in enumerate(distancias_canciones):
        if i < 10 or avg_dist <= radio:
            top_songs.append((song_id, votes))

    return top_songs




In [14]:
song_idx = 0
query_vectors = features_pca[song_idx * 20 : (song_idx + 1) * 20]
result = knn_top_R_tree(rtree_index, features_pca, track_ids, query_vectors, k=8) # Llamar a la función con k=3
resultado_final=[]
# Mostrar resultados
for i, (song_id, votes) in enumerate(result, 1):
    resultado_final.append(song_id)
print(resultado_final)

['7pse475uICmWRY5hEkvPvI', '5CwOUooch74h0XarhDfAQK', '3bZCS8ThTAxMJZavYWOY1z', '1U2xFfjK1QUuicENnW0iwv', '6MGryNr7aENIEfPUV1cHyg', '3OiEY2VLzrTyCoU8q2SQpe', '1oy6EH41CdAido7rIuuFzY', '2vPZ4Lklyu75zBR3SgbFNI']


In [32]:
# Indice LSH
# Librería faiss

import faiss
import numpy as np
import pickle
import time

# Cargar los datos
with open("feature_spotify.pkl", "rb") as file:
    track_ids = pickle.load(file)
    features = pickle.load(file)

# Validar que track_ids y features tengan el mismo tamaño
assert len(track_ids) == len(features), "track_ids y features no están alineados."

# Aplanar los vectores concatenando 20 vectores en un solo vector de dimensión 25600
features_flat = []
track_ids_flat = []

for idx, (track_id, feature_vectors) in enumerate(zip(track_ids, features)):
    if len(feature_vectors) == 20 and len(feature_vectors[0]) == 1280:  # Validar dimensión
        concatenated_vector = np.concatenate(feature_vectors)  # Concatenar los 20 vectores
        features_flat.append(concatenated_vector)  # Agregar al listado
        track_ids_flat.append(track_id)  # Agregar el track_id correspondiente

features_flat = np.array(features_flat, dtype="float32")  # Convertir a arreglo numpy

print(f"Número total de canciones: {len(features_flat)}")
print(f"Dimensión de cada vector concatenado: {features_flat.shape[1]}")  # Validar dimensión

# Crear el índice LSH
n_bits = 8192  # Número de bits para el hash LSH
n_bits = 16384  # Número de bits para el hash LSH
n_bits = len(features_flat*2)
index_lsh = faiss.IndexLSH(25600, n_bits)  # Actualizar dimensionalidad

# index_lsh.train(features_flat)  # Entrenar el índice LSH

# Indexación
start_time = time.time()
index_lsh.add(features_flat)
indexation_time = time.time() - start_time
print(f"Tiempo de indexación (LSH): {indexation_time:.2f} segundos")


Número total de canciones: 11883
Dimensión de cada vector concatenado: 25600
Tiempo de indexación (LSH): 27.30 segundos


In [33]:
print(features_flat.shape)

(11883, 25600)


In [34]:
# Función de búsqueda K-NN
def knn_search_lsh(query_index, k):
    query_vector = features_flat[query_index].reshape(1, -1)
    start_time = time.time()
    D, I = index_lsh.search(query_vector, k)
    search_time = time.time() - start_time
    
    # print(f"Búsqueda k-NN para el objeto en el índice {query_index} (k={k}):")
    # print("Distancias:", D[0])
    # print("Índices:", I[0])
    
    # Imprimir los track_ids correspondientes
    result_ids = [track_ids_flat[idx] for idx in I[0] if idx != -1]  # Ignorar índices inválidos
    # print("Track IDs:", result_ids)
    # print(f"Tiempo de búsqueda k-NN: {search_time:.2f} segundos")
    
    return result_ids


# Índice de consulta
query_index = 100 # Canción de consulta
query_index = 12 # Canción de consulta
k = 8       # Número de vecinos más cercanos

print("Buscar: ", track_ids_flat[query_index])

print()
# Ejecutar búsqueda K-NN
knn_res = knn_search_lsh(query_index, k)


Buscar:  4QtiVmuA88tPQiCOHZuQ5b



Parser


In [ ]:
import re
def parser(consulta):
    resultado_final=[]
    metodo = r"using\s+([a-zA-Z0-9_]+)" 
    song_id = r"where song_id\s*=\s*'([^']+)'"  
    top_k = r"LIMIT\s+([\d.]+)" 

    # Extraemos los valores usando `re.search`
    metodoknn = re.search(metodo, consulta)
    song_id_pre = re.search(song_id, consulta)
    pre_top_k = re.search(top_k, consulta)
    metodofinal = metodoknn.group(1) 
    song_id_final = song_id_pre.group(1)
    final_top_k = pre_top_k.group(1) 
    if metodofinal=='knn_top_R_tree':
        song_idx = int(song_id_final)
        query_vectors = features_pca[song_idx * 20 : (song_idx + 1) * 20]
        result = knn_top_R_tree(rtree_index, features_pca, track_ids, query_vectors, k=int(final_top_k)) # Llamar a la función con k=3
        # agregar los resultados
        for i, (song_id, votes) in enumerate(result, 1):
            resultado_final.append(song_id)
    if metodofinal=='knn_top_lineal':
        indicecancion = int(song_id_final)
        query = features_pca[indicecancion * 20 : (indicecancion + 1) * 20]  # consulta
        top_k =int(final_top_k)  #cuantos k vecinos cercanos quiero
        result = knn_top_lineal(features_pca, track_ids, query, top_k)
        for i, (song_id, votes) in enumerate(result, 1):
            resultado_final.append(song_id)
    if metodofinal=='knn_busquedarango_lineal':
        indicecancion = int(song_id_final)
        query = features_pca[indicecancion * 20 : (indicecancion + 1) * 20]
        radio = float(final_top_k) 
        result = knn_busquedarango_lineal(features_pca, track_ids, query,radio)
        for i, (song_id, votes) in enumerate(result, 1):
            resultado_final.append(song_id)
    if metodofinal=='knn_search_lsh':
        query_index = int(song_id_final)
        k = int(final_top_k) 
        resultado_final = knn_search_lsh(query_index, k)
        
    return resultado_final
    


consultas para utilizar:
"select song_id from spotify_songs using knn_top_lineal where song_id = '0' LIMIT 7"


"select song_id from spotify_songs using knn_top_R_tree where song_id = '0' LIMIT 7"


"select song_id from spotify_songs using knn_top_R_tree where song_id = '5' LIMIT 7"


"select song_id from spotify_songs using knn_top_lineal where song_id = '5' LIMIT 7"


"select song_id from spotify_songs using knn_busquedarango_lineal where song_id = '0' LIMIT 0.3"


"select song_id from spotify_songs using knn_top_R_tree where song_id = '12' LIMIT 5"


"select song_id from spotify_songs using knn_top_lineal where song_id = '12' LIMIT 5"


"select song_id from spotify_songs using knn_top_R_tree where song_id = '40' LIMIT 12"


"select song_id from spotify_songs using knn_top_lineal where song_id = '40' LIMIT 12"

In [20]:
consulta = "select song_id from spotify_songs using knn_top_R_tree where song_id = '12' LIMIT 5"
print(parser(consulta))


['4QtiVmuA88tPQiCOHZuQ5b', '2docvXwmea0pDCPv7cDe9s', '4Tvw0lweq9l2JPQKFbpbBQ', '0GNlKuzh7ZJTJ1iigkQEVp', '0JiVRyTJcJnmlwCZ854K4p']


In [21]:
consulta = "select song_id from spotify_songs using knn_top_lineal where song_id = '12' LIMIT 5"
print(parser(consulta))

['4QtiVmuA88tPQiCOHZuQ5b', '2docvXwmea0pDCPv7cDe9s', '0GNlKuzh7ZJTJ1iigkQEVp', '4Tvw0lweq9l2JPQKFbpbBQ', '52AWuNH7rgcyRdEM0lQzPz']


In [22]:
consulta = "select song_id from spotify_songs using knn_busquedarango_lineal where song_id = '12' LIMIT 5"
print(parser(consulta))

['4QtiVmuA88tPQiCOHZuQ5b', '2docvXwmea0pDCPv7cDe9s', '0GNlKuzh7ZJTJ1iigkQEVp', '4Tvw0lweq9l2JPQKFbpbBQ', '52AWuNH7rgcyRdEM0lQzPz', '0JiVRyTJcJnmlwCZ854K4p', '4wkWWZDqc7lgqOUA70pHMw', '6G8fblD9DbcEmaKOKDy3XL', '1AsRjeavWPH8sCQbjMP8Xh', '6TCk1yecQRDU5sXG3vjTYj', '0UZhP6BBkksOGmJ3hAABRD', '4LloVtxNZpeh7q7xdi1DQc', '7hvmAHUk63H4JBOQTxUwfx', '21xVx0n3qonzK2PXah9M1S', '7EEaV81pUNKvqsTrEHGoQH', '7FxzgizJRGTQ3fxUqfvljg', '2JEiLgAJl2jRakXjgAyATX', '1JhC0lqohQVWI3cQnHNtYM', '1QObH6m6jPyOHBLW2RLEpo', '79hH8RuVS9af6ixS97wREx', '1W7ZNYKcYjacT95mAVR6cS', '6AmhAYwQMKrZMZ5qTqxhli', '4QE7rcOeNErdz87pgSZ823', '3TIJkJgwWSAXb8zyOV6q5a', '6Zjw7NdXuLiwULHgyJ0Jk8', '53EbeKQwSawmvwdRsQyjRh', '37YWwmlRl1GonSwjRuOiEq', '4hAhyE1iL07EIUJnqw2bsV', '3UYiU57SMiAS5LqolhHJw1', '1lNn0udm3EHKFRCGpNKZxn', '3x6QCo3LClrqiJF19w8daE', '3JWIByW6qzV0IGsElrPkAj', '3otXXK647gcQHcy6Mu5aGy', '22gFNeEYuXuElY2QsLeMqx', '3Nbcx1svCSzmyYG2I81YCZ', '0NSeXLBOh16zjbENkAu0P6', '758wEKVqfYopJIHYWdLVd4', '2robgg0p9YjF9mc8NzhOSp', '3aoDEt6zSu

In [37]:
consulta = "select song_id from spotify_songs using knn_search_lsh where song_id = '12' LIMIT 5"
print(parser(consulta))

['4QtiVmuA88tPQiCOHZuQ5b', '4UAPxdGgrbJjht2mwHnMUu', '6q8onrzcka32nGVrnWYGBQ', '6WVheb5G29cbDOzF4qMfli', '7pyEd6g5y7SYdKNgRtleEB']
